In [1]:
from bancor_research import DEFAULT
from bancor_research.bancor_simulator.v3.spec.network import BancorDapp as sBancorDapp
from bancor_research.bancor_emulator.v3.spec.network import BancorDapp as eBancorDapp

# Recall from an earlier chapter that we already defined the whitelisted_tokens as follows.
whitelisted_tokens = {tkn_name : DEFAULT.WHITELIST[tkn_name] for tkn_name in DEFAULT.WHITELIST if tkn_name in ['eth', 'link', 'wbtc']}

# There are other possible configuration settings available, however for the present purpose we will use the defaults.
dapps = [BancorDapp(whitelisted_tokens=whitelisted_tokens) for BancorDapp in [sBancorDapp, eBancorDapp]]

def assertAndDisplay(decimals = -1):
    frames = [v3.describe(decimals) for v3 in dapps]
    diff = frames[0].compare(frames[1])
    assert diff.empty, diff
    return frames[0]

for v3 in dapps:
    v3.set_user_balance(tkn_amt='101', tkn_name='eth', user_name='Alice')
    v3.set_user_balance(tkn_amt='101', tkn_name='wbtc', user_name='Bob')
    v3.set_user_balance(tkn_amt='10001', tkn_name='link', user_name='Charlie')
    v3.set_user_balance(tkn_amt='2000', tkn_name='bnt', user_name='Trader')
    v3.set_user_balance(tkn_amt='1', tkn_name='eth', user_name='Trader')
    v3.set_user_balance(tkn_amt='3', tkn_name='link', user_name='Trader')

    v3.deposit(tkn_amt='100', tkn_name='eth', user_name='Alice')
    v3.deposit(tkn_amt='10000', tkn_name='link', user_name='Charlie')
    v3.deposit(tkn_amt='100', tkn_name='wbtc', user_name='Bob')
    v3.deposit(tkn_amt='1', tkn_name='eth', user_name='Alice')
    v3.deposit(tkn_amt='1', tkn_name='link', user_name='Charlie')
    v3.deposit(tkn_amt='1', tkn_name='wbtc', user_name='Bob')

    for tkn_name in whitelisted_tokens:
        v3.enable_trading(tkn_name)

# The Moving Average - EMA

A moving average is utilized as a security measure, where sudden changes in the pool reserves can be detected, and prevent abuse of the protocol's features. The moving average (ema) is updated with the first trade of the block, for any asset according to the following formula: 

![](https://lh5.googleusercontent.com/Hq0YWEHsrIza40yun3z0tuEu8migMiB6CzhIKZsY5TA2XCLw-zYqHGsm0J3QdB4-O1jxG1CF3swVjRyU4PTfQzyMJKxTCZRqS_D-9CbQYvcaZ-hh8_X2R_NGUqxIHigFGrpDeat7)

where r is the spot rate in units of BNT/TKN as determined by the trading liquidity balances of the pool, and α is an arbitrary constant that determines the responsiveness of the moving average. The α term is a global variable, set at 0.2 (or 20%) at launch of Bancor 3, and is intended to provide a consensus rate for the pool that is resistant to virtual price manipulation attacks. The following chart is an arbitrary depiction of the ema behavior relative to the spot price on a per-block basis. The ema is measured and updated before an action is executed; therefore, the ema response is delayed by a minimum of one action (e.g. a trade or add/remove liquidity event). Further, the ema is only adjusted once per pool, per block.

In [2]:
v3.describe(decimals=6)

bnt           eth          link  \
1 Account Alice                                0      0.000000             0   
          Bob                                  0             0             0   
          Charlie                              0             0      0.000000   
          Trader                     2000.000000      1.000000      3.000000   
2 Pool    a: TKN Staked Balance                0    101.000000  10001.000000   
          b: TKN Trading Liquidity             0     20.000000   3333.333333   
          c: BNT Trading Liquidity             0  20000.000000  20000.000000   
          d: BNT Current Funding               0  20000.000000  20000.000000   
          e: Spot Rate                         0   1000.000000      6.000000   
          f: Average Rate                      0   1000.000000      6.000000   
          g: Average Inverse Rate              0      0.001000      0.166667   
3 Network BNT Staked Balance        60000.000000             0             0   
          Master Vault              60000.000000    101.000000  10001.000000   
          Protection Vault              0.000000      0.000000      0.000000   
          Protocol Equity                      0             0             0   
          Rewards Vault                        0             0             0   

                                            wbtc         bnbnt       bneth  \
1 Account Alice                                0             0  101.000000   
          Bob                           0.000000             0           0   
          Charlie                              0             0           0   
          Trader                               0             0           0   
2 Pool    a: TKN Staked Balance       101.000000             0           0   
          b: TKN Trading Liquidity      1.250000             0           0   
          c: BNT Trading Liquidity  20000.000000             0           0   
          d: BNT Current Funding    20000.000000             0           0   
          e: Spot Rate              16000.000000             0           0   
          f: Average Rate           16000.000000             0           0   
          g: Average Inverse Rate       0.000062             0           0   
3 Network BNT Staked Balance                   0             0           0   
          Master Vault                101.000000             0           0   
          Protection Vault              0.000000             0           0   
          Protocol Equity                      0  60000.000000    0.000000   
          Rewards Vault                        0      0.000000    0.000000   

                                          bnlink      bnwbtc  vbnt  
1 Account Alice                                0           0   0.0  
          Bob                                  0  101.000000   0.0  
          Charlie                   10001.000000           0   0.0  
          Trader                               0           0   0.0  
2 Pool    a: TKN Staked Balance                0           0   0.0  
          b: TKN Trading Liquidity             0           0   0.0  
          c: BNT Trading Liquidity             0           0   0.0  
          d: BNT Current Funding               0           0   0.0  
          e: Spot Rate                         0           0   0.0  
          f: Average Rate                      0           0   0.0  
          g: Average Inverse Rate              0           0   0.0  
3 Network BNT Staked Balance                   0           0   0.0  
          Master Vault                         0           0   0.0  
          Protection Vault                     0           0   0.0  
          Protocol Equity               0.000000    0.000000   0.0  
          Rewards Vault                 0.000000    0.000000   0.0

At genesis, the ema rate is set equal to the spot rate. Therefore, in the above scenario each of the liquidity pools began with the following rates:

However, the trading situations do have an effect. To demonstrate, assume that the trades described above happen on consecutive blocks. For the LINK trading pool, both the spot rate and the ema begin at 6, as set at the genesis of the pool. Since the ema is adjusted before the trade is executed, the first trade has no effect on the ema; the spot price changes as expected. The new spot price becomes relevant in the next block, as the ema is updated prior to performing the second trade. First, the ema is updated using the new spot rate, then the second trade is processed. In this example, the lag of the ema means there is a significant gap between it and the spot price after the first block; however, the adjustment in the second block, prior to executing the second trade, results in a close agreement thereafter.

In [3]:
for v3 in dapps:
    v3.trade(tkn_amt='2000', source_token='bnt',  target_token='link', user_name='Trader', timestamp=0)
assertAndDisplay(decimals=4)

bnt         eth        link  \
1 Account Alice                         0.0000      0.0000      0.0000   
          Bob                           0.0000      0.0000      0.0000   
          Charlie                       0.0000      0.0000      0.0000   
          Trader                        0.0000      1.0000    303.0000   
2 Pool    a: TKN Staked Balance              0    101.0000  10003.4242   
          b: TKN Trading Liquidity           0     20.0000   3033.3333   
          c: BNT Trading Liquidity           0  20000.0000  21995.6044   
          d: BNT Current Funding             0  20000.0000  20000.0000   
          e: Spot Rate                       0   1000.0000      7.2513   
          f: Average Rate                    0   1000.0000      6.0000   
          g: Average Inverse Rate            0      0.0010      0.1667   
3 Network BNT Staked Balance        60000.0000           0           0   
          Master Vault              62000.0000    101.0000   9701.0000   
          Protection Vault              0.0000      0.0000      0.0000   
          Protocol Equity                    0           0           0   
          Rewards Vault                      0           0           0   

                                          wbtc       bnbnt     bneth  \
1 Account Alice                         0.0000      0.0000  101.0000   
          Bob                           0.0000      0.0000    0.0000   
          Charlie                       0.0000      0.0000    0.0000   
          Trader                        0.0000      0.0000    0.0000   
2 Pool    a: TKN Staked Balance       101.0000           0         0   
          b: TKN Trading Liquidity      1.2500           0         0   
          c: BNT Trading Liquidity  20000.0000           0         0   
          d: BNT Current Funding    20000.0000           0         0   
          e: Spot Rate              16000.0000           0         0   
          f: Average Rate           16000.0000           0         0   
          g: Average Inverse Rate       0.0001           0         0   
3 Network BNT Staked Balance                 0           0         0   
          Master Vault                101.0000           0         0   
          Protection Vault              0.0000           0         0   
          Protocol Equity                    0  60000.0000    0.0000   
          Rewards Vault                      0      0.0000    0.0000   

                                        bnlink    bnwbtc    vbnt  
1 Account Alice                         0.0000    0.0000  0.0000  
          Bob                           0.0000  101.0000  0.0000  
          Charlie                   10001.0000    0.0000  0.0000  
          Trader                        0.0000    0.0000  0.0000  
2 Pool    a: TKN Staked Balance              0         0       0  
          b: TKN Trading Liquidity           0         0       0  
          c: BNT Trading Liquidity           0         0       0  
          d: BNT Current Funding             0         0       0  
          e: Spot Rate                       0         0       0  
          f: Average Rate                    0         0       0  
          g: Average Inverse Rate            0         0       0  
3 Network BNT Staked Balance                 0         0       0  
          Master Vault                       0         0       0  
          Protection Vault                   0         0       0  
          Protocol Equity               0.0000    0.0000       0  
          Rewards Vault                 0.0000    0.0000       0

In [4]:
for v3 in dapps:
    v3.trade(tkn_amt='302.9981', source_token='link',  target_token='bnt', user_name='Trader', timestamp=1)
assertAndDisplay(decimals=4)

bnt         eth        link  \
1 Account Alice                         0.0000      0.0000      0.0000   
          Bob                           0.0000      0.0000      0.0000   
          Charlie                       0.0000      0.0000      0.0000   
          Trader                     1977.6153      1.0000      0.0019   
2 Pool    a: TKN Staked Balance              0    101.0000  10003.4242   
          b: TKN Trading Liquidity           0     20.0000   3336.3314   
          c: BNT Trading Liquidity           0  20000.0000  20013.9939   
          d: BNT Current Funding             0  20000.0000  20015.9807   
          e: Spot Rate                       0   1000.0000      5.9988   
          f: Average Rate                    0   1000.0000      6.2503   
          g: Average Inverse Rate            0      0.0010      0.1609   
3 Network BNT Staked Balance        60015.9807           0           0   
          Master Vault              60022.3847    101.0000  10003.9981   
          Protection Vault              0.0000      0.0000      0.0000   
          Protocol Equity                    0           0           0   
          Rewards Vault                      0           0           0   

                                          wbtc       bnbnt     bneth  \
1 Account Alice                         0.0000      0.0000  101.0000   
          Bob                           0.0000      0.0000    0.0000   
          Charlie                       0.0000      0.0000    0.0000   
          Trader                        0.0000      0.0000    0.0000   
2 Pool    a: TKN Staked Balance       101.0000           0         0   
          b: TKN Trading Liquidity      1.2500           0         0   
          c: BNT Trading Liquidity  20000.0000           0         0   
          d: BNT Current Funding    20000.0000           0         0   
          e: Spot Rate              16000.0000           0         0   
          f: Average Rate           16000.0000           0         0   
          g: Average Inverse Rate       0.0001           0         0   
3 Network BNT Staked Balance                 0           0         0   
          Master Vault                101.0000           0         0   
          Protection Vault              0.0000           0         0   
          Protocol Equity                    0  60000.0000    0.0000   
          Rewards Vault                      0      0.0000    0.0000   

                                        bnlink    bnwbtc    vbnt  
1 Account Alice                         0.0000    0.0000  0.0000  
          Bob                           0.0000  101.0000  0.0000  
          Charlie                   10001.0000    0.0000  0.0000  
          Trader                        0.0000    0.0000  0.0000  
2 Pool    a: TKN Staked Balance              0         0       0  
          b: TKN Trading Liquidity           0         0       0  
          c: BNT Trading Liquidity           0         0       0  
          d: BNT Current Funding             0         0       0  
          e: Spot Rate                       0         0       0  
          f: Average Rate                    0         0       0  
          g: Average Inverse Rate            0         0       0  
3 Network BNT Staked Balance                 0         0       0  
          Master Vault                       0         0       0  
          Protection Vault                   0         0       0  
          Protocol Equity               0.0000    0.0000       0  
          Rewards Vault                 0.0000    0.0000       0

In [5]:
for v3 in dapps:
    v3.trade(tkn_amt='1', source_token='eth',  target_token='wbtc', user_name='Trader', timestamp=2)
assertAndDisplay(decimals=4)

bnt         eth        link  \
1 Account Alice                         0.0000      0.0000      0.0000   
          Bob                           0.0000      0.0000      0.0000   
          Charlie                       0.0000      0.0000      0.0000   
          Trader                     1977.6153      0.0000      0.0019   
2 Pool    a: TKN Staked Balance              0    101.0000  10003.4242   
          b: TKN Trading Liquidity           0     21.0000   3336.3314   
          c: BNT Trading Liquidity           0  19055.2381  20013.9939   
          d: BNT Current Funding             0  20007.6190  20015.9807   
          e: Spot Rate                       0    907.3923      5.9988   
          f: Average Rate                    0   1000.0000      6.2503   
          g: Average Inverse Rate            0      0.0010      0.1609   
3 Network BNT Staked Balance        60023.5998           0           0   
          Master Vault              60022.3847    102.0000  10003.9981   
          Protection Vault              0.0000      0.0000      0.0000   
          Protocol Equity                    0           0           0   
          Rewards Vault                      0           0           0   

                                          wbtc       bnbnt     bneth  \
1 Account Alice                         0.0000      0.0000  101.0000   
          Bob                           0.0000      0.0000    0.0000   
          Charlie                       0.0000      0.0000    0.0000   
          Trader                        0.0557      0.0000    0.0000   
2 Pool    a: TKN Staked Balance       101.0005           0         0   
          b: TKN Trading Liquidity      1.1943           0         0   
          c: BNT Trading Liquidity  20940.8835           0         0   
          d: BNT Current Funding    20000.0000           0         0   
          e: Spot Rate              17534.2111           0         0   
          f: Average Rate           16000.0000           0         0   
          g: Average Inverse Rate       0.0001           0         0   
3 Network BNT Staked Balance                 0           0         0   
          Master Vault                100.9443           0         0   
          Protection Vault              0.0000           0         0   
          Protocol Equity                    0  60000.0000    0.0000   
          Rewards Vault                      0      0.0000    0.0000   

                                        bnlink    bnwbtc    vbnt  
1 Account Alice                         0.0000    0.0000  0.0000  
          Bob                           0.0000  101.0000  0.0000  
          Charlie                   10001.0000    0.0000  0.0000  
          Trader                        0.0000    0.0000  0.0000  
2 Pool    a: TKN Staked Balance              0         0       0  
          b: TKN Trading Liquidity           0         0       0  
          c: BNT Trading Liquidity           0         0       0  
          d: BNT Current Funding             0         0       0  
          e: Spot Rate                       0         0       0  
          f: Average Rate                    0         0       0  
          g: Average Inverse Rate            0         0       0  
3 Network BNT Staked Balance                 0         0       0  
          Master Vault                       0         0       0  
          Protection Vault                   0         0       0  
          Protocol Equity               0.0000    0.0000       0  
          Rewards Vault                 0.0000    0.0000       0